In [2]:
#lfp filter breakdown 


In [39]:
# run this cell
import pynwb
import os

#DataJoint and DataJoint schema
import datajoint as dj

## We also import a bunch of tables so that we can call them easily
#from nwb_datajoint.common import (RawPosition, HeadDir, Speed, LinPos, StateScriptFile, VideoFile,
#                                  DataAcquisitionDevice, CameraDevice, Probe,
#                                  DIOEvents,
#                                  ElectrodeGroup, Electrode, Raw, SampleCount,
#                                  LFPSelection, LFP, LFPBandSelection, LFPBand,
#                                  SortGroup, SpikeSorting, SpikeSorter, SpikeSorterParameters, SpikeSortingWaveformParameters, SpikeSortingParameters, SpikeSortingMetrics, CuratedSpikeSorting,\
#                                  FirFilter,
#                                  IntervalList, SortInterval,
#                                  Lab, LabMember, LabTeam, Institution,
#                                  BrainRegion,
#                                  SensorData,
#                                  Session, ExperimenterList,
#                                  Subject,
#                                  Task, TaskEpoch,
#                                  Nwbfile, AnalysisNwbfile, NwbfileKachery, AnalysisNwbfileKachery,
#                                  interval_list_contains,
#                                  interval_list_contains_ind,
#                                  interval_list_excludes,
#                                  interval_list_excludes_ind,
#                                  interval_list_intersect,
#                                  get_electrode_indices)

from nwb_datajoint.common import (RawPosition, HeadDir, Speed, LinPos, StateScriptFile, VideoFile,
                                  DataAcquisitionDevice, CameraDevice, Probe,
                                  DIOEvents,
                                  ElectrodeGroup, Electrode, Raw, SampleCount,
                                  LFPSelection, LFP, LFPBandSelection, LFPBand,
                                  SortGroup, SpikeSortingFilterParameters, SpikeSortingArtifactDetectionParameters,
                                  SpikeSortingRecordingSelection, SpikeSortingRecording, 
                                  SpikeSortingWorkspace, 
                                  SpikeSorter, SpikeSorterParameters, SortingID,
                                  SpikeSortingSelection, SpikeSorting, 
                                  SpikeSortingMetricParameters,
                                  ModifySortingParameters, ModifySortingSelection, ModifySorting, 
                                  AutomaticCurationParameters, AutomaticCurationSelection,
                                  AutomaticCuration,
                                  CuratedSpikeSortingSelection, CuratedSpikeSorting,
                                  UnitInclusionParameters,
                                  FirFilter,
                                  IntervalList, SortInterval,
                                  Lab, LabMember, LabTeam, Institution,
                                  BrainRegion,
                                  SensorData,
                                  Session, ExperimenterList,
                                  Subject,
                                  Task, TaskEpoch,
                                  Nwbfile, AnalysisNwbfile, NwbfileKachery, AnalysisNwbfileKachery,
                                 get_electrode_indices,IntervalList, interval_list_censor, interval_list_contains_ind, interval_list_intersect)  # noqa: F401)

# from .common_interval import 


import warnings
warnings.simplefilter('ignore', category=DeprecationWarning)
warnings.simplefilter('ignore', category=ResourceWarning)

In [14]:
from nwb_datajoint.common import get_electrode_indices
import numpy as np

In [11]:
nwb_file_name = 'beans20190718_emonroe_.nwb'
key=dict()
key['nwb_file_name'] = nwb_file_name


In [15]:
rawdata = Raw().nwb_object(key)
sampling_rate, interval_list_name = (Raw() & key).fetch1('sampling_rate', 'interval_list_name')
sampling_rate = int(np.round(sampling_rate))

valid_times = (IntervalList() & {'nwb_file_name': key['nwb_file_name'],'interval_list_name': interval_list_name}).fetch1('valid_times')
        # keep only the intervals > 1 second long
min_interval_length = 1.0
valid = []
for count, interval in enumerate(valid_times):
    if interval[1] - interval[0] > min_interval_length:
        valid.append(count)
valid_times = valid_times[valid] 
print(f'LFP: found {len(valid)} of {count+1} intervals > {min_interval_length} sec long.')  

LFP: found 5 of 5 intervals > 1.0 sec long.


In [17]:
decimation = sampling_rate // 1000
decimation

20

In [19]:
filter = (FirFilter() & {'filter_name': 'LFP 0-400 Hz'} &
                  {'filter_sampling_rate': sampling_rate}).fetch(as_dict=True)
filter

[{'filter_name': 'LFP 0-400 Hz',
  'filter_sampling_rate': 20000,
  'filter_type': 'lowpass',
  'filter_low_stop': 0.0,
  'filter_low_pass': 0.0,
  'filter_high_pass': 400.0,
  'filter_high_stop': 425.0,
  'filter_comments': 'standard LFP filter for 20 KHz data',
  'filter_band_edges': array([400, 425]),
  'filter_coeff': array([-2.75339220e-07, -1.09527344e-06, -1.89541262e-06, ...,
         -1.89541262e-06, -1.09527344e-06, -2.75339220e-07])}]

In [23]:
# there should only be one filter that matches, so we take the first of the dictionaries
key['filter_name'] = filter[0]['filter_name']
key['filter_sampling_rate'] = filter[0]['filter_sampling_rate']

filter_coeff = filter[0]['filter_coeff']
if len(filter_coeff) == 0:
    print(f'Error in LFP: no filter found with data sampling rate of {sampling_rate}')
print(filter_coeff)
print(key)

[-2.75339220e-07 -1.09527344e-06 -1.89541262e-06 ... -1.89541262e-06
 -1.09527344e-06 -2.75339220e-07]
{'nwb_file_name': 'beans20190718_emonroe_.nwb', 'filter_name': 'LFP 0-400 Hz', 'filter_sampling_rate': 20000}


In [25]:
# get the list of selected LFP Channels from LFPElectrode
electrode_keys = (LFPSelection.LFPElectrode & key).fetch('KEY')
electrode_id_list = list(k['electrode_id'] for k in electrode_keys)
electrode_id_list.sort()
electrode_id_list

[0, 1]

In [26]:
lfp_file_name = AnalysisNwbfile().create(key['nwb_file_name'])
lfp_file_name


Writing new NWB file beans20190718_emonroe_KQRK2EST3K.nwb


'beans20190718_emonroe_KQRK2EST3K.nwb'

In [27]:
#This is ther actual filtering step!! 
lfp_file_abspath = AnalysisNwbfile().get_abs_path(lfp_file_name)
lfp_object_id, timestamp_interval = FirFilter().filter_data_nwb(lfp_file_abspath, rawdata,
                                filter_coeff, valid_times, electrode_id_list, decimation)



Filtering data
Interval 0: loading data into memory
Interval 1: loading data into memory
Interval 2: loading data into memory
Interval 3: loading data into memory
Interval 4: loading data into memory


In [28]:
lfp_object_id

'e4937e2f-6e36-4356-af55-64e12bddee03'

In [29]:
timestamp_interval

[1563488988.8768327, 1563497107.134255]

In [35]:
# (AnalysisNwbfile() & {'nwb_file_name' : nwb_file_name}).fetch('analysis_parameters')

array([None], dtype=object)

In [40]:
# AnalysisNwbfile().add(key['nwb_file_name'], lfp_file_name)


key['analysis_file_name'] = lfp_file_name
key['lfp_object_id'] = lfp_object_id
key['lfp_sampling_rate'] = sampling_rate // decimation

        # finally, we need to censor the valid times to account for the downsampling
lfp_valid_times = interval_list_censor(valid_times, timestamp_interval)
        # add an interval list for the LFP valid times, skipping duplicates
key['interval_list_name'] = 'lfp valid times'
# IntervalList.insert1({'nwb_file_name': key['nwb_file_name'],
#                                'interval_list_name': key['interval_list_name'], 
#                                'valid_times': lfp_valid_times}, replace=True)
        # self.insert1(key)

In [41]:
key

{'nwb_file_name': 'beans20190718_emonroe_.nwb',
 'filter_name': 'LFP 0-400 Hz',
 'filter_sampling_rate': 20000,
 'analysis_file_name': 'beans20190718_emonroe_KQRK2EST3K.nwb',
 'lfp_object_id': 'e4937e2f-6e36-4356-af55-64e12bddee03',
 'lfp_sampling_rate': 1000,
 'interval_list_name': 'lfp valid times'}

In [ ]:
#     def make(self, key):
#         # get the NWB object with the data; FIX: change to fetch with additional infrastructure
#         rawdata = Raw().nwb_object(key)
#         sampling_rate, interval_list_name = (Raw() & key).fetch1(
#             'sampling_rate', 'interval_list_name')
#         sampling_rate = int(np.round(sampling_rate))

#         valid_times = (IntervalList() & {'nwb_file_name': key['nwb_file_name'],
#                                          'interval_list_name': interval_list_name}).fetch1('valid_times')
#         # keep only the intervals > 1 second long
#         min_interval_length = 1.0
#         valid = []
#         for count, interval in enumerate(valid_times):
#             if interval[1] - interval[0] > min_interval_length:
#                 valid.append(count)
#         valid_times = valid_times[valid] 
#         print(f'LFP: found {len(valid)} of {count+1} intervals > {min_interval_length} sec long.')  
        
#         # target 1 KHz sampling rate
#         decimation = sampling_rate // 1000

#         # get the LFP filter that matches the raw data
#         filter = (FirFilter() & {'filter_name': 'LFP 0-400 Hz'} &
#                   {'filter_sampling_rate': sampling_rate}).fetch(as_dict=True)

#         # there should only be one filter that matches, so we take the first of the dictionaries
#         key['filter_name'] = filter[0]['filter_name']
#         key['filter_sampling_rate'] = filter[0]['filter_sampling_rate']

#         filter_coeff = filter[0]['filter_coeff']
#         if len(filter_coeff) == 0:
#             print(
#                 f'Error in LFP: no filter found with data sampling rate of {sampling_rate}')
#             return None
#         # get the list of selected LFP Channels from LFPElectrode
#         electrode_keys = (LFPSelection.LFPElectrode & key).fetch('KEY')
#         electrode_id_list = list(k['electrode_id'] for k in electrode_keys)
#         electrode_id_list.sort()

#         lfp_file_name = AnalysisNwbfile().create(key['nwb_file_name'])

#         lfp_file_abspath = AnalysisNwbfile().get_abs_path(lfp_file_name)
#         lfp_object_id, timestamp_interval = FirFilter().filter_data_nwb(lfp_file_abspath, rawdata,
#                                                     filter_coeff, valid_times, electrode_id_list, decimation)

#         # now that the LFP is filtered and in the file, add the file to the AnalysisNwbfile table                                             
#         AnalysisNwbfile().add(key['nwb_file_name'], lfp_file_name)

#         key['analysis_file_name'] = lfp_file_name
#         key['lfp_object_id'] = lfp_object_id
#         key['lfp_sampling_rate'] = sampling_rate // decimation

#         # finally, we need to censor the valid times to account for the downsampling
#         lfp_valid_times = interval_list_censor(valid_times, timestamp_interval)
#         # add an interval list for the LFP valid times, skipping duplicates
#         key['interval_list_name'] = 'lfp valid times'
#         IntervalList.insert1({'nwb_file_name': key['nwb_file_name'],
#                                'interval_list_name': key['interval_list_name'], 
#                                'valid_times': lfp_valid_times}, replace=True)
#         self.insert1(key)

In [42]:
AnalysisNwbfile()

analysis_file_name name of the file,nwb_file_name name of the NWB file,analysis_file_abs_path the full path to the file,analysis_file_description an optional description of this analysis,analysis_parameters additional relevant parmeters. Currently used only for analyses
beans20190718_dgramling_5ZH6VUWBOC.nwb,beans20190718_dgramling_.nwb,=BLOB=,,=BLOB=
beans20190718_emonroe_XQVNQ0RWGJ.nwb,beans20190718_emonroe_.nwb,=BLOB=,,=BLOB=
CH5_20210109_2R15OCIEP3.nwb,CH5_20210109_.nwb,=BLOB=,,=BLOB=
CH5_20210109_3G6VVKKG9C.nwb,CH5_20210109_.nwb,=BLOB=,,=BLOB=
CH5_20210109_61G4ZVDIUH.nwb,CH5_20210109_.nwb,=BLOB=,,=BLOB=
CH5_20210109_6BCX2OOZUV.nwb,CH5_20210109_.nwb,=BLOB=,,=BLOB=
CH5_20210109_6FJG3QX7AW.nwb,CH5_20210109_.nwb,=BLOB=,,=BLOB=
CH5_20210109_6SS5QLIC0N.nwb,CH5_20210109_.nwb,=BLOB=,,=BLOB=
CH5_20210109_94CE7WQGCI.nwb,CH5_20210109_.nwb,=BLOB=,,=BLOB=
CH5_20210109_9RR98M30XR.nwb,CH5_20210109_.nwb,=BLOB=,,=BLOB=


In [43]:
LFP()

nwb_file_name name of the NWB file,interval_list_name descriptive name of this interval list,filter_name descriptive name of this filter,filter_sampling_rate sampling rate for this filter,analysis_file_name name of the file,lfp_object_id the NWB object ID for loading this object from the file,"lfp_sampling_rate the sampling rate, in HZ"
beans20190718_dgramling_.nwb,lfp valid times,LFP 0-400 Hz,20000,beans20190718_dgramling_5ZH6VUWBOC.nwb,35d1ec01-a384-48ff-abce-7f148c768220,1000.0
beans20190718_emonroe_.nwb,lfp valid times,LFP 0-400 Hz,20000,beans20190718_emonroe_XQVNQ0RWGJ.nwb,d95c48bd-b2d1-4cdd-a7ab-911fb917c15e,1000.0
CH6120211203_.nwb,lfp valid times,LFP 0-400 Hz,30000,CH6120211203_EMOR22N1Q2.nwb,b7f4ec79-c165-4974-9faf-3f21852b0339,1000.0
CH6520211201_.nwb,lfp valid times,LFP 0-400 Hz,30000,CH6520211201_0XPU9WSAU4.nwb,65bdf912-7e73-441c-8bf8-4bd085dae781,1000.0
despereaux20191125_.nwb,lfp valid times,LFP 0-400 Hz,30000,despereaux20191125_M1OH0TD4YH.nwb,1ea02648-f360-41fd-b412-d90c9ec9fd87,1000.0
J1620210529_.nwb,lfp valid times,LFP 0-400 Hz,30000,J1620210529_BQN7BHYVZ8.nwb,78880e9e-0b15-46cd-b2c2-ad089d0a0922,1000.0
J1620210531_.nwb,lfp valid times,LFP 0-400 Hz,30000,J1620210531_UKIY6CRTC2.nwb,665be634-59fa-47dd-9660-5369176e07b8,1000.0
J1620210612_.nwb,lfp valid times,LFP 0-400 Hz,30000,J1620210612_OSTPDQX6R0.nwb,3fab3179-58ea-4301-8656-ede2aa1f3b4a,1000.0
jaq20190815_.nwb,lfp valid times,LFP 0-400 Hz,30000,jaq20190815_BY2AYU4CG2.nwb,8969a43d-a7c3-4972-a78f-1d44c723c394,1000.0
montague20200731_.nwb,lfp valid times,LFP 0-400 Hz,30000,montague20200731_0N1626OC24.nwb,dc7015f0-381c-4a65-a071-62e16d9bcd3a,1000.0
